In [46]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from collections import deque
import matplotlib.pyplot as plt  # For visualization
import time
from utils import State, Action, get_all_valid_actions, change_state, is_terminal, terminal_utility

class UltimateTicTacToeNN(nn.Module):
    def __init__(self):
        super(UltimateTicTacToeNN, self).__init__()
        self.fc1 = nn.Linear(81, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return torch.sigmoid(self.fc3(x))

class MCTS:
    def __init__(self, model, simulations=50):
        self.model = model
        self.simulations = simulations
    
    def search(self, state):
        best_action = None
        best_value = -float('inf')
        valid_actions = get_all_valid_actions(state)
        
        for action in valid_actions:
            new_state = change_state(state, action)
            value = self.evaluate(new_state)
            if value > best_value:
                best_value = value
                best_action = action
        
        return best_action
    
    def evaluate(self, state):
        if is_terminal(state):
            return terminal_utility(state)
        
        board_flat = torch.tensor(state.board.flatten(), dtype=torch.float32)
        return self.model(board_flat).item()

class AlphaZeroTrainer:
    def __init__(self, model, lr=0.001, batch_size=32, memory_size=1000):
        self.model = model
        self.optimizer = optim.Adam(model.parameters(), lr=lr)
        self.criterion = nn.MSELoss()
        self.memory = deque(maxlen=memory_size)
        self.loss_history = []  # To track loss
        self.batch_size = batch_size
        self.game_history = []
    
    def store_experience(self, state, value):
        self.memory.append((state, value))
    
    def train(self):
        if len(self.memory) < self.batch_size:
            return
        
        batch = random.sample(self.memory, self.batch_size)
        states, values = zip(*batch)
        
        states_tensor = torch.stack([torch.tensor(s.board.flatten(), dtype=torch.float32) for s in states])
        values_tensor = torch.tensor(values, dtype=torch.float32).view(-1, 1)
        
        self.optimizer.zero_grad()
        predictions = self.model(states_tensor)
        loss = self.criterion(predictions, values_tensor)
        loss.backward()
        self.optimizer.step()
        self.loss_history.append(loss.item())  # Log the loss
    
    def self_play(self, games=500):
        for game in range(games):
            state = State()
            mcts = MCTS(self.model)
            history = []
            
            while not is_terminal(state):
                action = mcts.search(state)
                history.append((state, action))
                state = change_state(state, action)
            
            utility = terminal_utility(state)
            for s, _ in history:
                self.store_experience(s, utility)
            self.train()
            self.game_history.append(history)
            print(f"Game {game+1}/{games}: {self.loss_history}")

In [47]:
class MinimaxPlayer:
    def __init__(self, model, depth=5):
        self.model = model
        self.depth = depth
    
    def minimax(self, state, depth, maximizing, alpha, beta):
        # Check for terminal state or depth limit
        if depth == 0 or is_terminal(state):
            return self.evaluate(state)
        
        valid_actions = get_all_valid_actions(state)
        
        if maximizing:
            best_value = -float('inf')
            for action in valid_actions:
                new_state = change_state(state, action)
                value = self.minimax(new_state, depth - 1, False, alpha, beta)
                best_value = max(best_value, value)
                alpha = max(alpha, best_value)
                
                # Prune branches where beta <= alpha
                if beta <= alpha:
                    break
            
            return best_value
        else:
            best_value = float('inf')
            for action in valid_actions:
                new_state = change_state(state, action)
                value = self.minimax(new_state, depth - 1, True, alpha, beta)
                best_value = min(best_value, value)
                beta = min(beta, best_value)
                
                # Prune branches where beta <= alpha
                if beta <= alpha:
                    break
            
            return best_value
    
    def evaluate(self, state):
        # Get the model's evaluation of the board state
        board_flat = torch.tensor(state.board.flatten(), dtype=torch.float32)
        return self.model(board_flat).item()
    
    def get_best_move(self, state):
        best_action = None
        best_value = -float('inf')
        valid_actions = get_all_valid_actions(state)
        
        alpha = -float('inf')
        beta = float('inf')
        
        for action in valid_actions:
            new_state = change_state(state, action)
            value = self.minimax(new_state, self.depth, False, alpha, beta)
            if value > best_value:
                best_value = value
                best_action = action
        
        return best_action


In [28]:
model = UltimateTicTacToeNN()
trainer = AlphaZeroTrainer(model)
trainer.self_play(games=200)  # Train with self-play



Game 0/200: [0.21169781684875488]
Game 1/200: [0.21169781684875488, 0.18725381791591644]
Game 2/200: [0.21169781684875488, 0.18725381791591644, 0.20586471259593964]
Game 3/200: [0.21169781684875488, 0.18725381791591644, 0.20586471259593964, 0.1892216056585312]
Game 4/200: [0.21169781684875488, 0.18725381791591644, 0.20586471259593964, 0.1892216056585312, 0.19626733660697937]
Game 5/200: [0.21169781684875488, 0.18725381791591644, 0.20586471259593964, 0.1892216056585312, 0.19626733660697937, 0.21401214599609375]
Game 6/200: [0.21169781684875488, 0.18725381791591644, 0.20586471259593964, 0.1892216056585312, 0.19626733660697937, 0.21401214599609375, 0.14575159549713135]
Game 7/200: [0.21169781684875488, 0.18725381791591644, 0.20586471259593964, 0.1892216056585312, 0.19626733660697937, 0.21401214599609375, 0.14575159549713135, 0.1388973593711853]
Game 8/200: [0.21169781684875488, 0.18725381791591644, 0.20586471259593964, 0.1892216056585312, 0.19626733660697937, 0.21401214599609375, 0.145751

Game 27/200: [0.21169781684875488, 0.18725381791591644, 0.20586471259593964, 0.1892216056585312, 0.19626733660697937, 0.21401214599609375, 0.14575159549713135, 0.1388973593711853, 0.13349640369415283, 0.16142740845680237, 0.18165574967861176, 0.11923867464065552, 0.13849538564682007, 0.181874617934227, 0.14496399462223053, 0.1414038985967636, 0.1377202272415161, 0.16617336869239807, 0.19355972111225128, 0.18076011538505554, 0.17048361897468567, 0.12492464482784271, 0.18488697707653046, 0.15768222510814667, 0.16942796111106873, 0.1776256561279297, 0.18547746539115906, 0.19997385144233704]
Game 28/200: [0.21169781684875488, 0.18725381791591644, 0.20586471259593964, 0.1892216056585312, 0.19626733660697937, 0.21401214599609375, 0.14575159549713135, 0.1388973593711853, 0.13349640369415283, 0.16142740845680237, 0.18165574967861176, 0.11923867464065552, 0.13849538564682007, 0.181874617934227, 0.14496399462223053, 0.1414038985967636, 0.1377202272415161, 0.16617336869239807, 0.19355972111225128

Game 39/200: [0.21169781684875488, 0.18725381791591644, 0.20586471259593964, 0.1892216056585312, 0.19626733660697937, 0.21401214599609375, 0.14575159549713135, 0.1388973593711853, 0.13349640369415283, 0.16142740845680237, 0.18165574967861176, 0.11923867464065552, 0.13849538564682007, 0.181874617934227, 0.14496399462223053, 0.1414038985967636, 0.1377202272415161, 0.16617336869239807, 0.19355972111225128, 0.18076011538505554, 0.17048361897468567, 0.12492464482784271, 0.18488697707653046, 0.15768222510814667, 0.16942796111106873, 0.1776256561279297, 0.18547746539115906, 0.19997385144233704, 0.0779934823513031, 0.14633941650390625, 0.12992309033870697, 0.14035241305828094, 0.12732842564582825, 0.12441897392272949, 0.13277670741081238, 0.11891964077949524, 0.12029397487640381, 0.08584421128034592, 0.07405960559844971, 0.11881212145090103]
Game 40/200: [0.21169781684875488, 0.18725381791591644, 0.20586471259593964, 0.1892216056585312, 0.19626733660697937, 0.21401214599609375, 0.1457515954971

Game 48/200: [0.21169781684875488, 0.18725381791591644, 0.20586471259593964, 0.1892216056585312, 0.19626733660697937, 0.21401214599609375, 0.14575159549713135, 0.1388973593711853, 0.13349640369415283, 0.16142740845680237, 0.18165574967861176, 0.11923867464065552, 0.13849538564682007, 0.181874617934227, 0.14496399462223053, 0.1414038985967636, 0.1377202272415161, 0.16617336869239807, 0.19355972111225128, 0.18076011538505554, 0.17048361897468567, 0.12492464482784271, 0.18488697707653046, 0.15768222510814667, 0.16942796111106873, 0.1776256561279297, 0.18547746539115906, 0.19997385144233704, 0.0779934823513031, 0.14633941650390625, 0.12992309033870697, 0.14035241305828094, 0.12732842564582825, 0.12441897392272949, 0.13277670741081238, 0.11891964077949524, 0.12029397487640381, 0.08584421128034592, 0.07405960559844971, 0.11881212145090103, 0.08490394055843353, 0.16921986639499664, 0.17619451880455017, 0.21780677139759064, 0.16016344726085663, 0.11795894056558609, 0.10257365554571152, 0.15781

Game 57/200: [0.21169781684875488, 0.18725381791591644, 0.20586471259593964, 0.1892216056585312, 0.19626733660697937, 0.21401214599609375, 0.14575159549713135, 0.1388973593711853, 0.13349640369415283, 0.16142740845680237, 0.18165574967861176, 0.11923867464065552, 0.13849538564682007, 0.181874617934227, 0.14496399462223053, 0.1414038985967636, 0.1377202272415161, 0.16617336869239807, 0.19355972111225128, 0.18076011538505554, 0.17048361897468567, 0.12492464482784271, 0.18488697707653046, 0.15768222510814667, 0.16942796111106873, 0.1776256561279297, 0.18547746539115906, 0.19997385144233704, 0.0779934823513031, 0.14633941650390625, 0.12992309033870697, 0.14035241305828094, 0.12732842564582825, 0.12441897392272949, 0.13277670741081238, 0.11891964077949524, 0.12029397487640381, 0.08584421128034592, 0.07405960559844971, 0.11881212145090103, 0.08490394055843353, 0.16921986639499664, 0.17619451880455017, 0.21780677139759064, 0.16016344726085663, 0.11795894056558609, 0.10257365554571152, 0.15781

...

Game 186/200: [0.21169781684875488, 0.18725381791591644, 0.20586471259593964, 0.1892216056585312, 0.19626733660697937, 0.21401214599609375, 0.14575159549713135, 0.1388973593711853, 0.13349640369415283, 0.16142740845680237, 0.18165574967861176, 0.11923867464065552, 0.13849538564682007, 0.181874617934227, 0.14496399462223053, 0.1414038985967636, 0.1377202272415161, 0.16617336869239807, 0.19355972111225128, 0.18076011538505554, 0.17048361897468567, 0.12492464482784271, 0.18488697707653046, 0.15768222510814667, 0.16942796111106873, 0.1776256561279297, 0.18547746539115906, 0.19997385144233704, 0.0779934823513031, 0.14633941650390625, 0.12992309033870697, 0.14035241305828094, 0.12732842564582825, 0.12441897392272949, 0.13277670741081238, 0.11891964077949524, 0.12029397487640381, 0.08584421128034592, 0.07405960559844971, 0.11881212145090103, 0.08490394055843353, 0.16921986639499664, 0.17619451880455017, 0.21780677139759064, 0.16016344726085663, 0.11795894056558609, 0.10257365554571152, 0.1578

Game 189/200: [0.21169781684875488, 0.18725381791591644, 0.20586471259593964, 0.1892216056585312, 0.19626733660697937, 0.21401214599609375, 0.14575159549713135, 0.1388973593711853, 0.13349640369415283, 0.16142740845680237, 0.18165574967861176, 0.11923867464065552, 0.13849538564682007, 0.181874617934227, 0.14496399462223053, 0.1414038985967636, 0.1377202272415161, 0.16617336869239807, 0.19355972111225128, 0.18076011538505554, 0.17048361897468567, 0.12492464482784271, 0.18488697707653046, 0.15768222510814667, 0.16942796111106873, 0.1776256561279297, 0.18547746539115906, 0.19997385144233704, 0.0779934823513031, 0.14633941650390625, 0.12992309033870697, 0.14035241305828094, 0.12732842564582825, 0.12441897392272949, 0.13277670741081238, 0.11891964077949524, 0.12029397487640381, 0.08584421128034592, 0.07405960559844971, 0.11881212145090103, 0.08490394055843353, 0.16921986639499664, 0.17619451880455017, 0.21780677139759064, 0.16016344726085663, 0.11795894056558609, 0.10257365554571152, 0.1578

Game 193/200: [0.21169781684875488, 0.18725381791591644, 0.20586471259593964, 0.1892216056585312, 0.19626733660697937, 0.21401214599609375, 0.14575159549713135, 0.1388973593711853, 0.13349640369415283, 0.16142740845680237, 0.18165574967861176, 0.11923867464065552, 0.13849538564682007, 0.181874617934227, 0.14496399462223053, 0.1414038985967636, 0.1377202272415161, 0.16617336869239807, 0.19355972111225128, 0.18076011538505554, 0.17048361897468567, 0.12492464482784271, 0.18488697707653046, 0.15768222510814667, 0.16942796111106873, 0.1776256561279297, 0.18547746539115906, 0.19997385144233704, 0.0779934823513031, 0.14633941650390625, 0.12992309033870697, 0.14035241305828094, 0.12732842564582825, 0.12441897392272949, 0.13277670741081238, 0.11891964077949524, 0.12029397487640381, 0.08584421128034592, 0.07405960559844971, 0.11881212145090103, 0.08490394055843353, 0.16921986639499664, 0.17619451880455017, 0.21780677139759064, 0.16016344726085663, 0.11795894056558609, 0.10257365554571152, 0.1578

Game 196/200: [0.21169781684875488, 0.18725381791591644, 0.20586471259593964, 0.1892216056585312, 0.19626733660697937, 0.21401214599609375, 0.14575159549713135, 0.1388973593711853, 0.13349640369415283, 0.16142740845680237, 0.18165574967861176, 0.11923867464065552, 0.13849538564682007, 0.181874617934227, 0.14496399462223053, 0.1414038985967636, 0.1377202272415161, 0.16617336869239807, 0.19355972111225128, 0.18076011538505554, 0.17048361897468567, 0.12492464482784271, 0.18488697707653046, 0.15768222510814667, 0.16942796111106873, 0.1776256561279297, 0.18547746539115906, 0.19997385144233704, 0.0779934823513031, 0.14633941650390625, 0.12992309033870697, 0.14035241305828094, 0.12732842564582825, 0.12441897392272949, 0.13277670741081238, 0.11891964077949524, 0.12029397487640381, 0.08584421128034592, 0.07405960559844971, 0.11881212145090103, 0.08490394055843353, 0.16921986639499664, 0.17619451880455017, 0.21780677139759064, 0.16016344726085663, 0.11795894056558609, 0.10257365554571152, 0.1578

Game 198/200: [0.21169781684875488, 0.18725381791591644, 0.20586471259593964, 0.1892216056585312, 0.19626733660697937, 0.21401214599609375, 0.14575159549713135, 0.1388973593711853, 0.13349640369415283, 0.16142740845680237, 0.18165574967861176, 0.11923867464065552, 0.13849538564682007, 0.181874617934227, 0.14496399462223053, 0.1414038985967636, 0.1377202272415161, 0.16617336869239807, 0.19355972111225128, 0.18076011538505554, 0.17048361897468567, 0.12492464482784271, 0.18488697707653046, 0.15768222510814667, 0.16942796111106873, 0.1776256561279297, 0.18547746539115906, 0.19997385144233704, 0.0779934823513031, 0.14633941650390625, 0.12992309033870697, 0.14035241305828094, 0.12732842564582825, 0.12441897392272949, 0.13277670741081238, 0.11891964077949524, 0.12029397487640381, 0.08584421128034592, 0.07405960559844971, 0.11881212145090103, 0.08490394055843353, 0.16921986639499664, 0.17619451880455017, 0.21780677139759064, 0.16016344726085663, 0.11795894056558609, 0.10257365554571152, 0.1578

In [48]:
class StudentAgent:
    def __init__(self):
        """Instantiates your agent.
        """
        self.minimax_player = MinimaxPlayer(model)


    def choose_action(self, state: State) -> Action:
        """Returns a valid action to be played on the board.
        Assuming that you are filling in the board with number 1.

        Parameters
        ---------------
        state: The board to make a move on.
        """
        #initial_state = State()
        best_move = self.minimax_player.get_best_move(state)
        print("Best move from minimax evaluation:", best_move)
        return best_move

In [49]:
# Use this cell to test your agent in two full games against a random agent.
# The random agent will choose actions randomly among the valid actions.

class RandomStudentAgent(StudentAgent):
    def choose_action(self, state: State) -> Action:
        return state.get_random_valid_action()

def run(your_agent: StudentAgent, random_agent: StudentAgent, start_num: int):
    timeout_count = 0
    invalid_count = 0
    state = State(fill_num=start_num)
    while not state.is_terminal():
        random_action = state.get_random_valid_action()
        if state.fill_num == 2:
            action = random_agent.choose_action(state.clone())
        else:
            start_time = time.time()
            action = your_agent.choose_action(state.clone())
            end_time = time.time()
            if end_time - start_time > 3:
                print("Your agent time out!")
                timeout_count += 1
                action = random_action
        if not state.is_valid_action(action):
            assert state.fill_num == 1
            print("Your agent made an invalid action!")
            invalid_count += 1
            action = random_action
        state = state.change_state(action)
    if state.terminal_utility() == 1:
        print("You win!")
    elif state.terminal_utility() == 0:
        print("You lose!")
    else:
        print("Draw")
    print(f"Timeout count: {timeout_count}")
    print(f"Invalid count: {invalid_count}")

your_agent = StudentAgent()
random_agent = RandomStudentAgent()
run(your_agent, random_agent, 1)
run(your_agent, random_agent, 2)

KeyboardInterrupt: 

In [34]:
minimax_player = MinimaxPlayer(model)
minimax_player.evaluate(State())

0.6116111874580383

In [39]:
trainer.game_history[199]

[(State(
      board=
          0 0 0 | 0 0 0 | 0 0 0
          0 0 0 | 0 0 0 | 0 0 0
          0 0 0 | 0 0 0 | 0 0 0
          ---------------------
          0 0 0 | 0 0 0 | 0 0 0
          0 0 0 | 0 0 0 | 0 0 0
          0 0 0 | 0 0 0 | 0 0 0
          ---------------------
          0 0 0 | 0 0 0 | 0 0 0
          0 0 0 | 0 0 0 | 0 0 0
          0 0 0 | 0 0 0 | 0 0 0, 
      local_board_status=
          [[0 0 0]
           [0 0 0]
           [0 0 0]], 
      prev_local_action=None, 
      fill_num=1
  ),
  (2, 2, 2, 1)),
 (State(
      board=
          0 0 0 | 0 0 0 | 0 0 0
          0 0 0 | 0 0 0 | 0 0 0
          0 0 0 | 0 0 0 | 0 0 0
          ---------------------
          0 0 0 | 0 0 0 | 0 0 0
          0 0 0 | 0 0 0 | 0 0 0
          0 0 0 | 0 0 0 | 0 0 0
          ---------------------
          0 0 0 | 0 0 0 | 0 0 0
          0 0 0 | 0 0 0 | 0 0 0
          0 0 0 | 0 0 0 | 0 1 0, 
      local_board_status=
          [[0 0 0]
           [0 0 0]
           [0 0 0]], 
      

In [2]:
from utils import State
print(State())

State(
    board=
        0 0 0 | 0 0 0 | 0 0 0
        0 0 0 | 0 0 0 | 0 0 0
        0 0 0 | 0 0 0 | 0 0 0
        ---------------------
        0 0 0 | 0 0 0 | 0 0 0
        0 0 0 | 0 0 0 | 0 0 0
        0 0 0 | 0 0 0 | 0 0 0
        ---------------------
        0 0 0 | 0 0 0 | 0 0 0
        0 0 0 | 0 0 0 | 0 0 0
        0 0 0 | 0 0 0 | 0 0 0, 
    local_board_status=
        [[0 0 0]
         [0 0 0]
         [0 0 0]], 
    prev_local_action=None, 
    fill_num=1
)

